In [1]:
import pandas as pd

Всем привет! Поговорим про **join()** и **merge()** 

Основное _отличие_ join() от merge() в том, что join() производит склейку таблиц используя _индексы датафреймов_.
А в merge() мы можем сами задавать поля, которые будут использоваться для связи, причем можно использовать сразу несколько полей.

Возникает тогда вопрос: зачем вообще нужен join()? А дело вот в чем: join() может соединить несколько (более двух) датафреймов за одно действие. 

Создадим два датафрейма и будем смотреть, что с ними происходит :) 

In [2]:
d_1 = {'col1': ['Kate', 'Laina', 'Masha', 'Ann', 'Mikhail'], 
       'col2': ['хорошо', 'плохо', 'отлично', 'хорошо', 'ужасно']}
df_1 = pd.DataFrame(data = d_1)
df_1

,col1,col2
0,Kate,хорошо
1,Laina,плохо
2,Masha,отлично
3,Ann,хорошо
4,Mikhail,ужасно


In [3]:
d_2 = {'col1': ['Sasha', 'Ann', 'Mikhail', 'Valera', 'Max'], 'col3': [7, 4, 10, 6, 2]}
df_2 = pd.DataFrame(data = d_2)
df_2

,col1,col3
0,Sasha,7
1,Ann,4
2,Mikhail,10
3,Valera,6
4,Max,2


## **MERGE**

Запись следующая: 

**pd.merge(left, right, how = '', left_on = '', right_on = '')**

* left - датафрейм (название не обязательно left, может быть любое другое)

* right - датафрейм (название не обязательно right, может быть любое другое)

* how - каким способом будем склеивать датафреймы: 
    inner
    left
    right
  
* left_on  - по какому полю будем склеивать датафрейм left 

* right_on - по какому полю будем склеивать датафрейм right 
    
Если поля для соединения в обоих датафреймах имеют **одинаковые названия**, то лучше не использовать left_on и right_on, а написать один аргумент on

**pd.merge(left, right, how = '', on = '')**

In [4]:
pd.merge(df_1, df_2, how = 'inner', on = 'col1')

,col1,col2,col3
0,Ann,хорошо,4
1,Mikhail,ужасно,10


При необходимости склеить таблицы по нескольким полям синтаксис не изменяется, в left_on и right_on (или в on, если названия в двух датафреймах одинаковые) прописываем названия с помощью _[ ]_ (передаем список с названиями столбцов) и склеиваем таблицы :) 

**pd.merge(left, right, how = '', on = ['', ''])**

И так, усложним. В таблицах задублировалось значение по полю, используемого для присоединения. 

In [5]:
d_1_doubles = {'col1': ['Kate', 'Laina', 'Masha', 'Ann', 'Mikhail', 'Ann'], 
       'col2': ['хорошо', 'плохо', 'отлично', 'хорошо', 'ужасно', 'плохо'] }
df_1_doubles = pd.DataFrame(data = d_1_doubles)
df_1_doubles

,col1,col2
0,Kate,хорошо
1,Laina,плохо
2,Masha,отлично
3,Ann,хорошо
4,Mikhail,ужасно
5,Ann,плохо


In [6]:
d_2_doubles = {'col1': ['Sasha', 'Ann', 'Mikhail', 'Valera', 'Ann'], 'col3': [7, 4, 10, 6, 2]}
df_2_doubles = pd.DataFrame(data = d_2_doubles)
df_2_doubles

,col1,col3
0,Sasha,7
1,Ann,4
2,Mikhail,10
3,Valera,6
4,Ann,2


Склеиваем таблицы и получаем вот такое задвоение: 

In [7]:
pd.merge(df_1_doubles, df_2_doubles, how = 'inner', on = 'col1')

,col1,col2,col3
0,Ann,хорошо,4
1,Ann,хорошо,2
2,Ann,плохо,4
3,Ann,плохо,2
4,Mikhail,ужасно,10


Мы увидили самую типичную проблему при соединении таблиц. С подобным замножением строк вам придется столкнуться еще не один раз. Проблема возникает из-за того, что условие присоединения выполняется для нескольких строк в каждой таблице. 

В итоге к

| col1 | col2 |
| --- | --- |
| Ann | хорошо |

на третьей строчке присоединилось 

| col1 | col3 |
| --- | --- |
| Ann |	4 |
| Ann |	2 |	

тоже самое произошло для пятой строки  с 

| col1 | col2 |
| --- | --- |
| Ann | плохо |


**Проверяйте количество строк после присоединения. Если оно изменилось, убедитесь, что оно изменилось так, как вы этого хотели :)**




## **JOIN**

**Join** похож на merge, но для соединения таблиц мы можем использовать только их индексы или поля с одинаковыми названиями в присоединяемых датафреймах, поэтому рассмотрим только склейку более двух таблиц в одно действие по их индексам.

Вспомним наши таблицы:

In [8]:
df_1

,col1,col2
0,Kate,хорошо
1,Laina,плохо
2,Masha,отлично
3,Ann,хорошо
4,Mikhail,ужасно


In [9]:
df_2

,col1,col3
0,Sasha,7
1,Ann,4
2,Mikhail,10
3,Valera,6
4,Max,2


Создадим третью таблицу:

In [10]:
d_3 = {'col1': ['Kate', 'Laina', 'Masha', 'Ann', 'Mikhail'], 
       'col4': ['math', 'english', 'history', 'math', 'history']}
df_3 = pd.DataFrame(data = d_3)
df_3

,col1,col4
0,Kate,math
1,Laina,english
2,Masha,history
3,Ann,math
4,Mikhail,history


Назначим столбец **col1** индексом для всех таблиц:

In [11]:
df_1.set_index('col1', inplace=True)
df_2.set_index('col1', inplace=True)
df_3.set_index('col1', inplace=True)

После этой операции наши датафреймы выглядят таким образом:

In [12]:
df_2

,col3
col1,
Sasha,7
Ann,4
Mikhail,10
Valera,6
Max,2


Приступим к соединению:

In [13]:
df_1.join([df_2, df_3], how='inner')

,col2,col3,col4
col1,,,
Ann,хорошо,4,math
Mikhail,ужасно,10,history


Также есть еще одно отличие **join** и **merge**. 

- метод **merge** может быть вызвать как от pandas, так и от датафрейма: 

    `pd.merge(left, right, how = '', on = '')` 
    
    `left.merge(right, how = '', on = '')`
    
    
- метод **join** может быть вызван только от датафрейма:

    `left.join(right, how = '')`

## **Заключение**

В документации `Pandas` есть большой туториал по этим методам https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

Сейчас он может показаться местами не понятным, но здорово возвращаться к нему, когда появятся вопросы по склейке таблиц. Думаю, там есть примеры почти на все случаи жизни

И будьте внимательны с замножением строк :)